In [ ]:
import pandas as pd
import json
import os

In [ ]:
# import the AlboMP tweet
# I collected this at 10:09pm 09/04/2022 (AEST). Note - check how long this is since the tweet was authored.

list_liking_users_data_1512657668053540864 = []

with open('1512657668053540864_liking_users.jsonl') as old:
    for x in old:
        row = json.loads(x)
        # print(row)
        list_liking_users_data_1512657668053540864.append(pd.DataFrame(row["data"]))

In [ ]:
list_liking_users_data_1512657668053540864

In [ ]:
# combine all the data into one dataframe 
liking_users_1512657668053540864_df = pd.concat(list_liking_users_data_1512657668053540864)

In [ ]:
# view it 
liking_users_1512657668053540864_df

In [ ]:
# write to file
liking_users_1512657668053540864_df.to_csv('liking_users_1512657668053540864_df.csv')

# Botometer analysis of accounts

In [ ]:
import networkx as nx
import pandas as pd
import json
import botometer

In [ ]:
rapidapi_key = "xxx"

twitter_app_auth = {
    'consumer_key': 'xxx',
    'consumer_secret': 'xxx',
    'access_token': 'xxx',
    'access_token_secret': 'xxx',
  }

bom = botometer.Botometer(wait_on_ratelimit=True,
                          rapidapi_key=rapidapi_key,
                          **twitter_app_auth)

In [ ]:
all_users_ids_1512657668053540864_likers = list(liking_users_1512657668053540864_df['id'])

In [ ]:
all_users_ids_1512657668053540864_likers

In [ ]:
# we will append the results for each account to a master list
list_1512657668053540864_likers_bot_results = []

# check all account IDs
for x in range(len(all_users_ids_1512657668053540864_likers)):
    print("Checking: " + str(all_users_ids_1512657668053540864_likers[x]) + " ...")
    try:
        list_1512657668053540864_likers_bot_results.append(bom.check_account(int(all_users_ids_1512657668053540864_likers[x])))
    except:
        print("An error occurred! Skipping.")
        
print("How many users did we check in total:")
print(len(all_users_ids_1512657668053540864_likers))

print("How many did we collect data for (i.e. who weren't suspended)")
print(len(list_1512657668053540864_likers_bot_results))

In [ ]:
# construct dataframes for all users and store in master list, to then convert into dataframe

user_bot_data_ALL_list = []

for i in range(len(list_1512657668053540864_likers_bot_results)):
    # produce a single dataframe for a given user, combining all relevant data 
    df1_temp = pd.DataFrame([list_1512657668053540864_likers_bot_results[i]['user']['user_data']])
    df2_temp = pd.DataFrame([list_1512657668053540864_likers_bot_results[i]['display_scores']['english']])
    df3_temp = pd.DataFrame([list_1512657668053540864_likers_bot_results[i]['cap']])

    df1_temp['tmp'] = 1
    df2_temp['tmp'] = 1
    df3_temp['tmp'] = 1

    df_combined1 = pd.merge(df1_temp, df2_temp, on=['tmp'])

    df_combined2 = pd.merge(df_combined1,df3_temp, on=['tmp'])

    df_combined2.drop('tmp', axis=1, inplace=True)

    user_bot_data_ALL_list.append(df_combined2)

In [ ]:
# combine list of dataframes into single dataframe
df_all_users_botometer_results = pd.concat(user_bot_data_ALL_list)

df_all_users_botometer_results.rename(columns={'id_str': 'id'}, inplace=True)

df_all_users_botometer_results

In [ ]:
# write the data to file
df_all_users_botometer_results.to_csv('Botometer_results_1512657668053540864_likers.csv')

In [ ]:
# write account IDs to text file 
df_all_users_botometer_results['id'].to_csv(r'1512657668053540864_likers_account_Ids', header=None, index=None, sep='\n', mode='a')


In [ ]:
# Plot the universal scores
import seaborn as sns
import matplotlib.pyplot as plt

# set a grey background (use sns.set_theme() if seaborn version 0.11.0 or above) 
sns.set(style="darkgrid")
 
fig = sns.histplot(df_all_users_botometer_results['universal'])

# plt.gcf().set_size_inches(12, 8)
plt.gcf().set_size_inches(20, 10)
fig.tick_params(axis='both', which='major', labelsize=20)

fig.set_xlabel("Bot score", fontsize = 30)
fig.set_ylabel("Number of Twitter accounts", fontsize = 30)
plt.title("Botometer scores for likers of AlboMP's tweet (tweet ID: 1512657668053540864)", fontdict={'fontsize':20})

plt.show()

In [ ]:
df_all_users_botometer_results[df_all_users_botometer_results['universal'] > 0.95]

In [ ]:
len(df_all_users_botometer_results[df_all_users_botometer_results['universal'] > 0.95])

In [ ]:
len(df_all_users_botometer_results[df_all_users_botometer_results['universal'] > 0.95]) / len(df_all_users_botometer_results)

In [ ]:
df_all_users_botometer_results[df_all_users_botometer_results['universal'] == 1.0]

In [ ]:
# set a grey background (use sns.set_theme() if seaborn version 0.11.0 or above) 
sns.set(style="darkgrid")
df = sns.load_dataset('iris')
 
# plotting both distibutions on the same figure
fig = sns.kdeplot(df_all_users_botometer_results['astroturf'], shade=True)
fig = sns.kdeplot(df_all_users_botometer_results['fake_follower'], shade=True)
fig = sns.kdeplot(df_all_users_botometer_results['financial'], shade=True)
fig = sns.kdeplot(df_all_users_botometer_results['self_declared'], shade=True)
fig = sns.kdeplot(df_all_users_botometer_results['spammer'], shade=True)

plt.gcf().set_size_inches(12, 8)
fig.legend(labels=['astroturf','fake_follower','financial','self_declared','spammer'])

fig.set_xlabel("Score", fontsize = 20)
fig.set_ylabel("Density", fontsize = 20)

plt.show()

In [ ]:
# analyse self-declared accounts
df_all_users_botometer_results[df_all_users_botometer_results['self_declared'] > 4]

In [ ]:
pd.set_option('display.max_rows', None)
df_all_users_botometer_results[df_all_users_botometer_results['universal'] > 0.99]